In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from core import system_msg, user_msg, assistant_msg

In [3]:
from sampling import fetch_joined_data
df = await fetch_joined_data()
df = df.sort_values(['meeting_timestamp'])

In [4]:
from elastic_search import VectorSearchEngine
search_engine = VectorSearchEngine(device=None)
search_engine.create_index(df)


Using device: cpu
Creating field-specific indices...
Processing topic_name...


  0%|          | 0/400 [00:00<?, ?it/s]

Processing summary...


  0%|          | 0/400 [00:00<?, ?it/s]

Processing details...


  0%|          | 0/400 [00:00<?, ?it/s]

In [5]:
meeting_speaker_sets = df.groupby('meeting_id')['speaker_name'].agg(set)
meeting_speaker_sets = meeting_speaker_sets.apply(frozenset)
meeting_combinations = meeting_speaker_sets.value_counts()
print(meeting_combinations.head(20))

speaker_name
(Olga Nemirovskaya, Dmitriy Grankin)                                    33
(Dmitriy Grankin)                                                       32
(nan)                                                                   20
(Sergey Ryabenko, Dmitriy Grankin)                                      15
(nan (1), nan (0))                                                       6
(Dmitriy Grankin, Andrey Pisankin)                                       4
(Sergey Ryabenko, Dmitriy Grankin, Nikolay)                              3
(Dmitriy Grankin, Alex Shevliakov)                                       2
(nan (1), nan, nan (0))                                                  2
(Dmitriy Grankin, Matt Lewis)                                            2
(Dmitriy Grankin, Ilia Semukhin)                                         2
(Alex Shevliakov, Dmitry Grankin)                                        2
(Rick Tousseyn, Dmitriy Grankin)                                         2
(Sergey Ryab

In [6]:
team = ['Dmitry Grankin','Olga Nemirovskaya','Dmitriy Grankin','Sergey Ryabenko','Alex Shevliakov','Karine Stepanyan','Andrey Pisankin',
        'Okafor Ikenna','nan']

In [7]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [8]:
search_engine.get_search_hints("product ", max_hints=5)

['Product Hunt',
 'Product Hunter',
 'Product School',
 'product hunter',
 'Product Managers']

In [9]:
from IPython.display import Markdown
from core import system_msg, user_msg, generic_call_stream

In [10]:
from prompts import Prompts
prompts = Prompts()


In [11]:
class ChatSession:
    def __init__(self, search_engine, prompts):
        self.search_engine = search_engine
        self.prompts = prompts
        self.messages = []
    
    def add_message(self, role, content):
        # Convert to Msg object instead of dict
        if role == "system":
            self.messages.append(system_msg(content))
        elif role == "user":
            self.messages.append(user_msg(content))
        elif role == "assistant":
            self.messages.append(assistant_msg(content))
    
    async def process_user_message(self, query):

        self.add_message("user", query)
        query, instructions = query.split(':') if ':' in query else (query, '')
        
        # Search for context
        main_results = self.search_engine.search(
            query=query,
            k=200,
            min_similarity=0.4,
            exact_match_boost=0.3,
            return_scores=True
        )
        speaker_results = self.search_engine.search_by_speaker(
            query,
            k=200,
            min_similarity=0.49,
            return_scores=True
        )
        
        # Process results
        main_results = main_results[['topic_name','speaker_name','summary','details']] if len(main_results) > 0 else pd.DataFrame()
        speaker_results = speaker_results[['topic_name','speaker_name','summary','details']] if len(speaker_results) > 0 else pd.DataFrame()
        self.results = pd.concat([main_results, speaker_results]).drop_duplicates().reset_index(drop=True)
        context = self.results[['speaker_name','topic_name','summary','details']].to_markdown(index=False)
        
        # Add context message
        context_msg = system_msg(f"Context: {context}")
        
        # Generate response
        messages_with_context = [
            system_msg(self.prompts.perplexity),
            system_msg('special instructions: '+instructions),
            *self.messages,
            context_msg
        ]
        
        response = await generic_call_stream(messages_with_context)
        
        # Add assistant response
        self.add_message("assistant", response)
        
        return response

In [12]:

chat = ChatSession(search_engine, prompts)

In [13]:
chat.messages

[]

In [14]:

response = await chat.process_user_message("dashboard")
Markdown(response)

The dashboard is a critical feature of the VEX product, serving as a user interface that organizes and presents information in an easily readable format. It allows users to interact with various functionalities, track user engagement, and manage meeting summaries and action items. The team is actively working on enhancing the dashboard's usability, addressing issues such as thread naming, button functionality, and the need for filtering options to improve user experience[1][2][3][4][5][6][7][8][9][10].

Recent discussions have highlighted the importance of user feedback in guiding the dashboard's development, with plans to implement features that facilitate better navigation and data visibility. The dashboard is also undergoing updates to ensure it collects necessary data, including cookies for tracking user interactions[11][12][13][14][15][16][17][18][19][20].


The dashboard is a critical feature of the VEX product, serving as a user interface that organizes and presents information in an easily readable format. It allows users to interact with various functionalities, track user engagement, and manage meeting summaries and action items. The team is actively working on enhancing the dashboard's usability, addressing issues such as thread naming, button functionality, and the need for filtering options to improve user experience[1][2][3][4][5][6][7][8][9][10].

Recent discussions have highlighted the importance of user feedback in guiding the dashboard's development, with plans to implement features that facilitate better navigation and data visibility. The dashboard is also undergoing updates to ensure it collects necessary data, including cookies for tracking user interactions[11][12][13][14][15][16][17][18][19][20].

In [82]:
chat.results

""


In [22]:
response = await chat.process_user_message("Alexey: more people")

Alexey
 more people
Alexey is involved in several key activities related to project development and promotion:

1. **New Build Development**: Alexey is responsible for providing a new build aimed at resolving issues with the extension, particularly concerning button functionality. This build is crucial for the next steps in the project, although no specific deadline has been mentioned[1][3].

2. **Promotion through Network**: Olga Nemirovskaya suggested engaging Alexey to leverage his network for promoting Vexu, as he is currently leading the project. This could enhance the reach of their promotional efforts[2].

3. **UI Development**: Alexey is also a developer focused on creating the user interface for the project. His work is being reviewed by Dmitriy Grankin, who is discussing backend requirements and the overall functionality of the assistant[3][4].

4. **Insights into Web3**: Alexey Khintz, another individual mentioned, is noted as a contact in the Web3 space who could provide va

In [55]:
chat.results

,topic_name,speaker_name,summary,details
0,Alex Shevliakov,Alex Shevliakov,Alex Shevliakov is a speaker in the meeting who is working on a startup and his own projects.,"Alex shares his current work situation, mentioning he is trying to minimize freelance work and is focused on developing his projects. He discusses the importance of user feedback in product development."
1,Alex Shevliakov,Alex Shevliakov,"Alex Shevliakov is a speaker in the meeting, contributing to discussions on dashboard functionalities and user experience.","Alex discussed various features for the dashboard, including filtering options and the assistant's capabilities."
2,Alex Shevliakov,Alex Shevliakov,"Alex Shevliakov is a speaker in the meeting, contributing to discussions about the interface and layout issues.","Alex is involved in addressing the design and usability of the interface, particularly in relation to buttons and highlights."
3,Alex Shevliakov,Alex Shevliakov,"Alex Shevliakov is a speaker in the meeting who discusses work hours, project management, and coding practices.",He shares his experiences with work hours and the challenges of debugging and testing in software development.
4,Alex Shevliakov,Alex Shevliakov,Alex Shevliakov is a speaker in the meeting who discusses project timelines and component integration.,"He is focused on delivering components for a dashboard and integrating server logic, while also managing his time effectively."
...,...,...,...,...
199,HTML code,Olga Nemirovskaya,HTML code is discussed in the context of inserting necessary scripts into the Tilda platform.,The team discusses where to place the HTML code within the head section of the website settings.
200,Insert HTML code into the head section of the website,Olga Nemirovskaya,Olga Nemirovskaya will insert the necessary HTML code into the head section of the website to ensure proper functionality.,Olga will access the project settings in Tilda and insert the provided HTML code into the head section. This task is important for integrating necessary scripts and tracking functionalities.
201,Olga Nemirovskaya,Olga Nemirovskaya,"Olga Nemirovskaya is a participant in the meeting, assisting with technical changes in the Tilda platform.",She is asked to make changes in Tilda regarding domain settings and HTML code insertion.
202,B2B,Olga Nemirovskaya,B2B refers to business-to-business marketing strategies and approaches.,The team discusses the need for a separate B2B page to cater to business clients.


In [54]:
response = await chat.process_user_message("how Xut tuX relevant to vexa technology stack")

how Xut tuX relevant to vexa technology stack

Xut tuX's background and experience are highly relevant to Vexa's technology stack, particularly in the areas of backend development and voice processing. His expertise includes:

- **Backend Development**: Xut has significant experience in backend technologies, particularly with Python and Rust. This aligns with Vexa's need for robust backend systems to handle real-time audio processing and transcription.

- **Voice Bot Technology**: Xut has worked on projects involving voice bots and call center automation, which directly relates to Vexa's focus on transcribing spoken words into text and providing real-time assistance during meetings. His experience with automating call handling for clients like Почта России and Unicredit Bank showcases his capability in developing systems that enhance communication, similar to Vexa's objectives.

- **Database Management**: His work with Postgres, including implementing physical replication for real-time

In [56]:
chat.messages

[Msg(role='user', content='Xut tuX technology stack', stage=None, service_content=None),
 Msg(role='assistant', content="Xut tuX's technology stack primarily includes:\n\n- **Programming Languages**: Xut has experience with Python and is currently transitioning to Rust for backend development, particularly in high-frequency trading applications.\n\n- **Database Technologies**: He works with Postgres, implementing physical replication for real-time data updates and backups. Redis is also mentioned as part of his projects for data management.\n\n- **Containerization and Deployment**: Xut utilizes Docker for containerization, enhancing deployment and scalability of applications. He has also worked with Nginx for managing containerized applications.\n\n- **Asynchronous Task Management**: Celery is part of his stack for handling asynchronous tasks in backend development.\n\n- **DevOps Practices**: Xut has experience with DevOps methodologies, focusing on collaboration between development an

In [15]:
chat.messages

[Msg(role='user', content='', stage=None, service_content=None),
 Msg(role='assistant', content='The discussions surrounding marketing strategies for Vexa and its products highlight a multifaceted approach involving various platforms and community engagement methods. \n\nGoogle Ads is a central focus, with multiple team members, including Dmitriy Grankin and Olga Nemirovskaya, emphasizing its role in user acquisition and advertising effectiveness. The team is actively setting up campaigns, analyzing performance data, and planning targeted ads to attract users, particularly in relation to competitors and user needs[1][2][3][4][5]. \n\nCommunity engagement is also a significant theme, with suggestions to utilize platforms like Reddit and Telegram for outreach and feedback collection. Olga Nemirovskaya advocates for building a community around Vexa to enhance user experience and retention, while Dmitriy Grankin stresses the importance of active participation in relevant communities to fos